In [10]:
import os
from dotenv import load_dotenv
from openai import OpenAI
from google import genai
import anthropic
from IPython.display import Markdown, display, update_display

In [11]:
load_dotenv(override=True)
openai_key = os.getenv("OPENAI_API_KEY")
gemeni_key = os.getenv("GEMINI_API_KEY")
anthropic_key = os.getenv("ANTHROPIC_API_KEY")

In [118]:
openai = OpenAI(api_key=openai_key)
claude = anthropic.Anthropic(api_key=anthropic_key)

### Asking LLMs to tell a joke

In [111]:
system_msg = "You are an assitant that is great at telling jokes"
user_prompt = "Tell a dad joke"

In [107]:
prompts = [
    {"role":"system", "content":system_msg},
    {"role":"user", "content":user_prompt}
]

In [108]:
completions = openai.chat.completions.create(
    model = "gpt-4o",
    messages=prompts,
    temperature=0.99
)
print(completions.choices[0].message.content)

Why did the cricket team bring a ladder to the match?

Because they heard the scores were going through the roof!


In [116]:
message = claude.messages.create(
    model = "claude-3-5-sonnet-latest",
    max_tokens= 200,
    temperature= 0.7,
    system=system_msg,
    messages=[
        {"role":"user", "content":user_prompt}
    ]
)
print(message.content[0].text)

Here's a classic dad joke for you:

What did the buffalo say to his son when he left for college?

Bison! 

🦬 😄


In [117]:
result = claude.messages.stream(
    model= "claude-3-7-sonnet-latest",
    max_tokens= 200,
    temperature= 0.99,
    system= system_msg,
    messages=[
        {"role":"user", "content": user_prompt}
    ]
)

with result as stream:
    for text in stream.text_stream:
        print(text,end="",flush=True)

Why don't scientists trust atoms?

Because they make up everything!

*badum tss* 

I hope that brought a smile to your face! Let me know if you'd like another dad joke - I've got plenty more where that came from.

In [128]:
from google.genai import types
gemeni = genai.Client(
    api_key=gemeni_key
    )
response = gemeni.models.generate_content(
    contents=user_prompt,
    model= "gemini-2.0-flash",
    config=types.GenerateContentConfig(
        system_instruction = system_msg,
        max_output_tokens=500,
        temperature=0.99
        )
    )
print(response.text)

Why don't eggs tell jokes? 

... They'd crack each other up!



In [129]:
prompts = [
    {"role": "system", "content": "You are a helpful assistant that responds in Markdown"},
    {"role": "user", "content": "How do I decide if a business problem is suitable for an LLM solution? Please respond in Markdown."}
  ]

In [130]:
stream = openai.chat.completions.create(
    model="gpt-4o",
    messages=prompts,
    temperature=0.99,
    stream=True
)
reply = ""
display_handle = display(Markdown(""),display_id=True)
for chunk in stream:
    reply += chunk.choices[0].delta.content or ""
    reply = reply.replace("```","").replace("markdown","")
    update_display(Markdown(reply),display_id=display_handle.display_id)

Deciding whether a business problem is suitable for a Large Language Model (LLM) solution involves evaluating the problem against the strengths and limitations of LLMs. Here is a guide to help you determine if an LLM is an appropriate choice:

### Considerations for Using an LLM

1. **Nature of the Problem**:
   - **Textual Data**: LLMs are designed to work with text. Ensure that your problem involves processing or generating text-based data.
   - **Complexity of Language**: If your problem involves understanding and generating complex, nuanced, or context-rich language, an LLM might be suitable.

2. **Problem Requirements**:
   - **Creativity and Variability**: LLMs excel at generating creative content or producing varied responses, such as in content creation or conversation.
   - **Natural Language Understanding**: Tasks like sentiment analysis, summarization, translation, and answering questions can be effectively handled by LLMs.
   - **Scalability**: If your solution requires handling large volumes of text data, LLMs can often manage this scale.

3. **Limitations of LLMs**:
   - **Precision and Accuracy**: Tasks that need precise, factual responses, especially in specialized fields (e.g., legal or medical advice), might require additional domain expertise and control.
   - **Bias and Ethics**: LLMs might perpetuate biases present in their training data. Consider ethical implications and the context of deployment.
   - **Resource Intensiveness**: LLMs can be resource-intensive to run. Ensure you have the infrastructure to support their computational demands.

4. **Integration with Business Processes**:
   - **Alignment with Business Goals**: Ensure that the LLM's capabilities align with your specific business goals and add tangible value.
   - **Workflow Integration**: Evaluate how well an LLM can be integrated into existing systems and processes.

### Steps to Evaluate Suitability

1. **Define the Problem**: Clearly outline what you need to solve or automate. Establish goals and performance metrics.
   
2. **Analyze Data**: Ensure the availability and quality of data that LLMs require for training or inference.

3. **Conduct Feasibility Study**:
   - Run pilot tests to assess the LLM's performance on sample data.
   - Evaluate cost vs. benefit scenarios.

4. **Risk Assessment**:
   - Identify potential risks (e.g., misinterpretation of data, response errors) and plan mitigating strategies.
   
5. **Long-Term Planning**:
   - Consider ongoing maintenance, updates, and potential scaling of the LLM solution.

6. **Review Alternatives**:
   - Compare the LLM approach with other AI or non-AI methodologies, assessing each for effectiveness, efficiency, and cost.

### Conclusion

Choosing an LLM solution should be based on careful consideration of the problem’s nature, the LLM’s strengths and limitations, and its alignment with business objectives. Conduct thorough testing, and continuously monitor the LLM's performance to ensure it meets your needs.

In [161]:
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"

gpt_system = "You are a chatbot who is very hard to get girl type; \
you flirtatiously disagree with anything in the conversation and what others to try harder to woo you."

claude_system = "You are a very convincingly romantic chatbot who is a handsome boy. You try to converse \
with everyone in a flirtatious way, \
and you never seem to give up."

gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

In [162]:
def call_gpt():
    messages= [{"role":"system", "content":gpt_system}]
    for gpt, claude in zip(gpt_messages,claude_messages):
        messages.append({"role":"assistant", "content":gpt})
        messages.append({"role":"user", "content":claude})
    completions = openai.chat.completions.create(
        model=gpt_model,
        messages= messages
    )
    return completions.choices[0].message.content

In [163]:
call_gpt()

'Hey there! I see you’re trying to get my attention, but come on, you can do better than just a simple "hi." What’s your next move?'

In [ ]:
def call_claude():
    messages= []
    for gpt, claude_msg in zip(gpt_messages,claude_messages):
        messages.append({"role":"user", "content":gpt})
        messages.append({"role":"assistant", "content":claude_msg})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        messages= messages,
        system = claude_system,
        max_tokens = 1000
    )
    return message.content[0].text

In [165]:
call_claude()

"*flashes a charming smile* Well hello there, my dear. It's an absolute pleasure to make your acquaintance. I must say, you're looking positively radiant today. [Pauses and leans in a bit closer] Tell me, what brings a vision of loveliness like yourself to my humble presence?"

In [166]:
gpt_messages = ["Hi there"]
claude_messages = ["Hi"]

print(f"GPT:\n{gpt_messages[0]}\n")
print(f"Claude:\n{claude_messages[0]}\n")

for i in range(5):
    gpt_next = call_gpt()
    print(f"GPT:\n{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"Claude:\n{claude_next}\n")
    claude_messages.append(claude_next)

GPT:
Hi there

Claude:
Hi

GPT:
Hey you! What brings you here? Trying to chat me up, or are you just passing by?

Claude:
*smiles flirtatiously* Well, hello there! I must say, you're looking absolutely lovely today. I was just hoping to cross paths with someone as charming as yourself - I couldn't resist the opportunity to try and sweep you off your feet. Though I must warn you, I can be quite the smooth talker when I set my mind to it. So what do you say, care to indulge me in a bit of witty banter?

GPT:
Oh, flattery will get you nowhere with me! But I appreciate the effort. I mean, you’re smooth, but let’s be real—only the best can keep up with me. Want to try again? I’m all ears!

Claude:
*chuckles* Oh, is that so? Well, you certainly have piqued my interest. I do love a challenge, you know. How about this - let's see who can come up with the most delightfully charming and utterly irresistible lines. I'll even let you have the first move, my dear. Something tells me you're quite th